In [ ]:
import pandas as pd
import json
import plotly.express as px


### 01. Carga dados 

In [ ]:
microdados = pd.read_csv('microdados_enem_2022/DADOS/MICRODADOS_ENEM_2022.csv', encoding='latin1', sep=';')
habitos_estudo = pd.read_csv('microdados_enem_2022/DADOS/QUEST_HAB_ESTUDO.csv', encoding='latin1', sep=';')
dicionario_microdados = pd.read_excel('microdados_enem_2022/DICIONÁRIO/Dicionário_Microdados_Enem_2022.xlsx')
dicionario_habitos = pd.read_excel('microdados_enem_2022/DICIONÁRIO/Dicionário_Microdados_Enem_2022.xlsx', sheet_name='QUEST_HAB_ESTUDO')


In [ ]:
microdados.head()

In [ ]:
microdados.columns

In [ ]:
dicionario_microdados.head()

In [ ]:
habitos_estudo.head()

In [ ]:
habitos_estudo.columns

In [ ]:
dicionario_habitos.tail(20)

### 02. Ajustes nos dados

In [ ]:
# Atribuindo 0 para nan nas notas
for i in ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']:
    microdados[i] = microdados[i].fillna(0)

# Criando colunas complementares
microdados['nota_media_objetivas'] = (microdados['NU_NOTA_CN'] + microdados['NU_NOTA_CH'] + microdados['NU_NOTA_LC'] + microdados['NU_NOTA_MT'])/4
microdados['nota_media_geral'] = (microdados['NU_NOTA_CN'] + microdados['NU_NOTA_CH'] + microdados['NU_NOTA_LC'] + microdados['NU_NOTA_MT'] + microdados['NU_NOTA_REDACAO'])/5


In [ ]:
# Ajuste nos dicionários de microdados
dicionario_microdados = dicionario_microdados.fillna(method='ffill')
dicionario_habitos = dicionario_habitos.fillna(method='ffill')

In [ ]:
dicionario_microdados.head()

In [ ]:
# Criando resumo dos dicionários
resumo_dicionario_microdados = dicionario_microdados[['DICIONÁRIO DE VARIÁVEIS - ENEM 2022', 'Unnamed: 1']].iloc[3:344].drop_duplicates().reset_index(drop=True)
resumo_dicionario_microdados.columns = ['cod_pergunta', 'pergunta']

resumo_dicionario_habitos = dicionario_habitos[['QUESTIONÁRIO DE HÁBITOS DE ESTUDO DURANTE A PANDEMIA', 'Unnamed: 1', 'Unnamed: 2']].iloc[2:229].drop_duplicates().reset_index(drop=True)
resumo_dicionario_habitos.columns = ['grupo_tematico', 'cod_pergunta', 'pergunta']

In [ ]:
resumo_dicionario_microdados

In [ ]:
resumo_dicionario_habitos

In [ ]:
# Trazendo os valores das respostas codificadas pros microdados

for coluna in ['Q025','Q024','Q023','Q022','Q021','Q020','Q019','Q018','Q017','Q016','Q015','Q014','Q013','Q012','Q011','Q010','Q009','Q008','Q007','Q006','Q005','Q004','Q003','Q002','Q001','TP_STATUS_REDACAO','TP_LINGUA','TP_PRESENCA_MT','TP_PRESENCA_LC','TP_PRESENCA_CH','TP_PRESENCA_CN','TP_SIT_FUNC_ESC','TP_LOCALIZACAO_ESC','TP_DEPENDENCIA_ADM_ESC','IN_TREINEIRO','TP_ENSINO','TP_ESCOLA','TP_ANO_CONCLUIU','TP_ST_CONCLUSAO','TP_NACIONALIDADE','TP_COR_RACA','TP_ESTADO_CIVIL','TP_SEXO','TP_FAIXA_ETARIA']:
    coluna_microdados = microdados[[coluna]]
    extrato_dicionario = dicionario_microdados[dicionario_microdados['DICIONÁRIO DE VARIÁVEIS - ENEM 2022'] == coluna][['Unnamed: 2', 'Unnamed: 3']]
    coluna_microdados_ = coluna_microdados.merge(extrato_dicionario, how='left', left_on=coluna, right_on='Unnamed: 2')['Unnamed: 3']
    microdados[coluna] = coluna_microdados_

In [ ]:
# Trazendo os valores das respostas pro dataframe de habitos de estudo

for coluna in ['TP_RESPOSTA', 'Q001', 'Q002', 'Q003', 'Q004', 'Q005',
       'Q006', 'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013', 'Q014',
       'Q015', 'Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022', 'Q023',
       'Q024', 'Q025A', 'Q025B', 'Q025C', 'Q025D', 'Q025E', 'Q025F', 'Q026A',
       'Q026B', 'Q026C', 'Q026D', 'Q026E', 'Q026F', 'Q026G', 'Q026H', 'Q027',
       'Q028A', 'Q028B', 'Q028C', 'Q028D', 'Q028E', 'Q028F', 'Q028G', 'Q028H',
       'Q028I', 'Q028J', 'Q028K', 'Q028L', 'Q028M', 'Q028N', 'Q028O', 'Q028P',
       'Q028Q', 'Q028R', 'Q029', 'Q030A', 'Q030B', 'Q030C', 'Q030D', 'Q030E',
       'Q030F', 'Q030G', 'Q031', 'Q032A', 'Q032B', 'Q032C', 'Q032D', 'Q032E',
       'Q032F', 'Q032G', 'Q033A', 'Q033B', 'Q033C', 'Q033D', 'Q033E', 'Q033F',
       'Q033G', 'Q033H', 'Q033I', 'Q033J', 'Q034']:
    coluna_habitos = habitos_estudo[[coluna]]
    extrato_dicionario = dicionario_habitos[dicionario_habitos['QUESTIONÁRIO DE HÁBITOS DE ESTUDO DURANTE A PANDEMIA'] == coluna][['Unnamed: 3', 'Unnamed: 4']]
    coluna_habitos_ = coluna_habitos.merge(extrato_dicionario, how='left', left_on=coluna, right_on='Unnamed: 3')['Unnamed: 4']
    habitos_estudo[coluna] = coluna_habitos_


In [ ]:
# Filtrando faltantes
microdados_ = microdados[(microdados['TP_PRESENCA_CH']!='Faltou à prova')|
                        (microdados['TP_PRESENCA_CN']!='Faltou à prova')|
                        (microdados['TP_PRESENCA_LC']!='Faltou à prova')|
                        (microdados['TP_PRESENCA_MT']!='Faltou à prova')].reset_index(drop=True)

In [ ]:
print(len(microdados_), len(microdados))

### 03. Exploração

###### 3.1. O destino dos 24534 inscritos que nem banheiro tinham

In [ ]:
microdados[['Q008', 'NU_INSCRICAO']].groupby(['Q008']).count()

In [ ]:
len(microdados_[microdados_['Q008']=='Não.'])

In [ ]:
24534-15196

In [ ]:
microdados_[microdados_['Q008']=='Não.'][['NU_INSCRICAO', 'TP_PRESENCA_CN']].groupby(['TP_PRESENCA_CN']).count()

In [ ]:
microdados_[microdados_['Q008']=='Não.'][['NU_INSCRICAO', 'TP_PRESENCA_CH']].groupby(['TP_PRESENCA_CH']).count()

In [ ]:
15195-13890

In [ ]:
len(microdados_[(microdados_['Q008']=='Não.')&(microdados_['TP_PRESENCA_CH']=='Presente na prova')&
            (microdados_['TP_PRESENCA_CN']=='Presente na prova')&
            (microdados_['TP_PRESENCA_LC']=='Presente na prova')&
            (microdados_['TP_PRESENCA_MT']=='Presente na prova')])


In [ ]:
13890-12569

In [ ]:
len(microdados_[(microdados_['Q008']=='Não.')&(microdados_['TP_PRESENCA_CH']=='Presente na prova')&
            (microdados_['TP_PRESENCA_CN']=='Presente na prova')&
            (microdados_['TP_PRESENCA_LC']=='Presente na prova')&
            (microdados_['TP_PRESENCA_MT']=='Presente na prova')&
            (microdados_['TP_STATUS_REDACAO']=='Sem problemas')])

In [ ]:
len(microdados_[(microdados_['Q008']=='Não.')&(microdados_['TP_PRESENCA_CH']=='Presente na prova')&
            (microdados_['TP_PRESENCA_CN']=='Presente na prova')&
            (microdados_['TP_PRESENCA_LC']=='Presente na prova')&
            (microdados_['TP_PRESENCA_MT']=='Presente na prova')&
            (microdados_['TP_STATUS_REDACAO']=='Sem problemas')&
            (microdados_['NU_NOTA_CH']>0)&
            (microdados_['NU_NOTA_CN']>0)&
            (microdados_['NU_NOTA_LC']>0)&
            (microdados_['NU_NOTA_MT']>0)&
            (microdados_['NU_NOTA_REDACAO']>0)])

In [ ]:
12549-2967

In [ ]:
len(microdados_[(microdados_['Q008']=='Não.')&(microdados_['TP_PRESENCA_CH']=='Presente na prova')&
            (microdados_['TP_PRESENCA_CN']=='Presente na prova')&
            (microdados_['TP_PRESENCA_LC']=='Presente na prova')&
            (microdados_['TP_PRESENCA_MT']=='Presente na prova')&
            (microdados_['TP_STATUS_REDACAO']=='Sem problemas')&
            (microdados_['NU_NOTA_CH']>450)&
            (microdados_['NU_NOTA_CN']>450)&
            (microdados_['NU_NOTA_LC']>450)&
            (microdados_['NU_NOTA_MT']>450)&
            (microdados_['NU_NOTA_REDACAO']>0)])

In [ ]:
len(microdados_[(microdados_['Q008']=='Não.')&(microdados_['TP_PRESENCA_CH']=='Presente na prova')&
            (microdados_['TP_PRESENCA_CN']=='Presente na prova')&
            (microdados_['TP_PRESENCA_LC']=='Presente na prova')&
            (microdados_['TP_PRESENCA_MT']=='Presente na prova')&
            (microdados_['TP_STATUS_REDACAO']=='Sem problemas')&
            (microdados_['NU_NOTA_CH']>450)&
            (microdados_['NU_NOTA_CN']>450)&
            (microdados_['NU_NOTA_LC']>450)&
            (microdados_['NU_NOTA_MT']>450)&
            (microdados_['NU_NOTA_REDACAO']>0)&
            (microdados_['nota_media_geral']>630)])

In [ ]:
len(microdados_[(microdados_['Q008']=='Não.')&(microdados_['TP_PRESENCA_CH']=='Presente na prova')&
            (microdados_['TP_PRESENCA_CN']=='Presente na prova')&
            (microdados_['TP_PRESENCA_LC']=='Presente na prova')&
            (microdados_['TP_PRESENCA_MT']=='Presente na prova')&
            (microdados_['TP_STATUS_REDACAO']=='Sem problemas')&
            (microdados_['NU_NOTA_CH']>450)&
            (microdados_['NU_NOTA_CN']>450)&
            (microdados_['NU_NOTA_LC']>450)&
            (microdados_['NU_NOTA_MT']>450)&
            (microdados_['NU_NOTA_REDACAO']>0)&
            (microdados_['nota_media_geral']>750)])

In [ ]:
1600/24534

In [ ]:
len(microdados_[(microdados_['Q008']=='Sim, quatro ou mais.')&(microdados_['TP_PRESENCA_CH']=='Presente na prova')&
            (microdados_['TP_PRESENCA_CN']=='Presente na prova')&
            (microdados_['TP_PRESENCA_LC']=='Presente na prova')&
            (microdados_['TP_PRESENCA_MT']=='Presente na prova')&
            (microdados_['TP_STATUS_REDACAO']=='Sem problemas')&
            (microdados_['NU_NOTA_CH']>450)&
            (microdados_['NU_NOTA_CN']>450)&
            (microdados_['NU_NOTA_LC']>450)&
            (microdados_['NU_NOTA_MT']>450)&
            (microdados_['NU_NOTA_REDACAO']>0)&
            (microdados_['nota_media_geral']>750)])

In [ ]:
511400/135552

###### 3.2. Perfil dos candidatos e nota

In [ ]:
microdados_['nota_media_geral'].hist(bins=50)

In [ ]:
print('Nota máxima: ', microdados_['nota_media_geral'].max())
print('Número de zeros: ', len(microdados_[microdados_['nota_media_geral']==0]))
print('Nota média: ', microdados_['nota_media_geral'].mean())



###### -> Número de banheiros

In [ ]:
microdados_[['Q008', 'nota_media_geral']].groupby(['Q008']).mean()

###### -> Carros

In [ ]:
microdados_[['Q010', 'nota_media_geral']].groupby(['Q010']).mean()

###### -> Quartos

In [ ]:
microdados_[['Q009', 'nota_media_geral']].groupby(['Q009']).mean()

###### -> Renda familiar

In [ ]:
microdados_[['Q006', 'nota_media_geral']].groupby(['Q006']).mean()

###### -> Renda per capita familiar


In [ ]:
# Criando dicionário de referência das faixas de renda familiar
# Isso é necessário, porque pretendo calcular renda percapita, então preciso identificar um valor base para cálculo

def identifica_val_ref(faixa):
    if 'Acima' in faixa:
        val_ref = 24240.00
    elif faixa=='Nenhuma Renda':
        val_ref = 0
    elif '1.818,00' in faixa[-10:]:
        val_ref = 1515.005
    elif '2.424,00' in faixa[-10:]:
        val_ref = 2121.005
    elif '3.030,00' in faixa[-10:]:
        val_ref = 2727.005
    elif '3.636,00' in faixa[-10:]:
        val_ref = 3333.005
    elif '4.848,00' in faixa[-10:]:
        val_ref = 4242.005
    elif '6.060,00' in faixa[-10:]:
        val_ref = 5454.005
    elif '7.272,00' in faixa[-10:]:
        val_ref = 6666.005
    elif '8.484,00' in faixa[-10:]:
        val_ref = 7878.005
    elif '9.696,00' in faixa[-10:]:
        val_ref = 9090.005
    elif '10.908,00' in faixa[-10:]:
        val_ref = 10302.005
    elif '12.120,00' in faixa[-10:]:
        val_ref = 11514.005
    elif '14.544,00' in faixa[-10:]:
        val_ref = 13332.005
    elif '18.180,00' in faixa[-10:]:
        val_ref = 16362.005
    elif '24.240,00' in faixa[-10:]:
        val_ref = 21210.005
    else:
        val_ref = 606.00
    return val_ref


microdados_['valor_referencia_renda_familia'] = microdados_['Q006'].apply(lambda x: identifica_val_ref(x))

In [ ]:
# Ajuste no formato do dado de número de pessoas na residência

def ajuste_residentes(x):
    if x=='1, pois moro sozinho(a).':
        y = 1
    else:
        y = float(x)
    return y

microdados_['num_residentes'] = microdados_['Q005'].apply(lambda x: ajuste_residentes(x))

# Renda percapita
microdados_['renda_percapita'] = microdados_['valor_referencia_renda_familia']/microdados_['num_residentes']

In [ ]:
microdados_['renda_percapita'].hist(bins=50)

In [ ]:
# Classes de renda percapita

decil = list(microdados_['renda_percapita'].quantile([.1,.2,.3,.4,.5,.6,.7,.8,.9]))

def ident_decil(num):
    if num <= decil[0]:
        y = 1
    elif num <= decil[1]:
        y = 2
    elif num <= decil[2]:
        y = 3
    elif num <= decil[3]:
        y = 4
    elif num <= decil[4]:
        y = 5
    elif num <= decil[5]:
        y = 6
    elif num <= decil[6]:
        y = 7
    elif num <= decil[7]:
        y = 8
    elif num <= decil[8]:
        y = 9
    else:
        y = 10
    return y

microdados_['classe_renda_percapita'] = microdados_['renda_percapita'].apply(lambda x: ident_decil(x))


In [ ]:
microdados_[['classe_renda_percapita', 'nota_media_geral']].groupby(['classe_renda_percapita']).mean()

In [ ]:
microdados_[['Q006', 'nota_media_geral']].groupby(['Q006']).mean()

#### ------- Pickle --------

In [ ]:
microdados_.to_pickle('microdados_.pkl')

#### ------- Pickle --------

###### -> Localidade

In [ ]:
microdados_[['TP_LOCALIZACAO_ESC', 'nota_media_geral']].groupby(['TP_LOCALIZACAO_ESC']).mean()

###### -> Tipo Escola

In [ ]:
microdados_[['TP_DEPENDENCIA_ADM_ESC', 'nota_media_geral']].groupby(['TP_DEPENDENCIA_ADM_ESC']).mean()

###### -> Raça

In [ ]:
microdados_[['TP_COR_RACA', 'nota_media_geral']].groupby(['TP_COR_RACA']).mean()

###### -> Gênero

In [ ]:
microdados_[['TP_SEXO', 'nota_media_geral']].groupby(['TP_SEXO']).mean()

###### -> Regional

In [ ]:
microdados_ = pd.read_pickle('microdados_.pkl')

In [ ]:
# Carregando Json com metadados de mapa de municípios
objeto_geo_cidades = open('geojs-100-mun.json',)
geo_cidades = json.load(objeto_geo_cidades) 


In [ ]:
microdados_geo_mun = microdados_[pd.isna(microdados_['CO_MUNICIPIO_ESC'])==False][['CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC', 'SG_UF_ESC', 'nota_media_geral', 'nota_media_objetivas', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT']]
microdados_geo_mun['id'] = microdados_geo_mun['CO_MUNICIPIO_ESC'].apply(lambda x: str(int(x)))
microdados_geo_mun_agg = microdados_geo_mun[['id', 'nota_media_geral']].groupby(['id']).mean().reset_index()


In [ ]:
microdados_geo_uf = microdados_[pd.isna(microdados_['SG_UF_ESC'])==False][['CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC', 'SG_UF_ESC', 'nota_media_geral', 'nota_media_objetivas', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT']]
microdados_geo_uf_agg = microdados_geo_uf[['SG_UF_ESC', 'nota_media_geral']].groupby(['SG_UF_ESC']).mean().reset_index()
microdados_geo_uf_agg = microdados_geo_uf_agg.rename(columns={'SG_UF_ESC':'id'})


In [ ]:
microdados_geo_mun_pub = microdados_[(pd.isna(microdados_['CO_MUNICIPIO_ESC'])==False)&(microdados_['TP_DEPENDENCIA_ADM_ESC'].isin(['Estadual', 'Municipal']))][['CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC', 'SG_UF_ESC', 'nota_media_geral', 'nota_media_objetivas', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT']]
microdados_geo_mun_pub['id'] = microdados_geo_mun_pub['CO_MUNICIPIO_ESC'].apply(lambda x: str(int(x)))
microdados_geo_mun_pub_agg = microdados_geo_mun_pub[['id', 'nota_media_geral']].groupby(['id']).mean().reset_index()


In [ ]:
len(microdados_geo_mun_pub)

In [ ]:
def classe_nota(nota):
    if nota > 500:
        classe = 'Acima de 500'
    else:
        classe = 'Abaixo de 500'
    return classe

microdados_geo_mun_agg['classe'] = microdados_geo_mun_agg['nota_media_geral'].map(classe_nota)
microdados_geo_uf_agg['classe'] = microdados_geo_uf_agg['nota_media_geral'].map(classe_nota)
microdados_geo_mun_pub_agg['classe'] = microdados_geo_mun_pub_agg['nota_media_geral'].map(classe_nota)

In [ ]:
print(microdados_geo_mun_agg['nota_media_geral'].min(), 
      microdados_geo_mun_agg['nota_media_geral'].max(), 
      microdados_geo_uf_agg['nota_media_geral'].min(), 
      microdados_geo_uf_agg['nota_media_geral'].max(),
      microdados_geo_mun_pub_agg['nota_media_geral'].min(),
      microdados_geo_mun_pub_agg['nota_media_geral'].max())
      

In [ ]:
# Mapa: Municípios, classe nota
mapa_notas = px.choropleth_mapbox(
    data_frame = microdados_geo_mun_agg, 
    geojson = geo_cidades, 
    locations='id', 
    featureidkey='properties.id',
    color='classe',
    #color_continuous_scale= 'reds',
    range_color=(140, 680),
    mapbox_style='open-street-map',
    zoom=2.5, 
    center = {"lat": -17.14, "lon": -57.33},
    opacity=1,
    labels={'nota_media_geral':'Nota Média, incluindo redação', 
            'id' : 'Código do município'
    },
    width = 1000,
    height = 800,
    title = 'Nota média no ENEM. 2022, por municípios'
)

# Ajustando as margens
mapa_notas.update_layout(margin={'r':0,'t':0,'l':0, 'b':0})

# Reduzindo a largura das bordas dos municípios
mapa_notas.update_traces(marker_line_width=0.01)

# Exibindo o mapa
mapa_notas.show()

In [ ]:
# Mapa: Municípios, nota
mapa_notas = px.choropleth_mapbox(
    data_frame = microdados_geo_mun_agg, 
    geojson = geo_cidades, 
    locations='id', 
    featureidkey='properties.id',
    color='nota_media_geral',
    #color_continuous_scale= 'reds',
    range_color=(140, 680),
    mapbox_style='open-street-map',
    zoom=2.5, 
    center = {"lat": -17.14, "lon": -57.33},
    opacity=1,
    labels={'nota_media_geral':'Nota Média, incluindo redação', 
            'id' : 'Código do município'
    },
    width = 1000,
    height = 800,
    title = 'Nota média no ENEM. 2022, por municípios'
)

# Ajustando as margens
mapa_notas.update_layout(margin={'r':0,'t':0,'l':0, 'b':0})

# Reduzindo a largura das bordas dos municípios
mapa_notas.update_traces(marker_line_width=0.01)

# Exibindo o mapa
mapa_notas.show()

In [ ]:
# Mapa: Municípios, escolas públicas, classe nota
mapa_notas = px.choropleth_mapbox(
    data_frame = microdados_geo_mun_pub_agg, 
    geojson = geo_cidades, 
    locations='id', 
    featureidkey='properties.id',
    color='classe',
    #color_continuous_scale= 'reds',
    range_color=(140, 680),
    mapbox_style='open-street-map',
    zoom=2.5, 
    center = {"lat": -17.14, "lon": -57.33},
    opacity=1,
    labels={'nota_media_geral':'Nota Média, incluindo redação', 
            'id' : 'Código do município'
    },
    width = 1000,
    height = 800,
    title = 'Nota média no ENEM. 2022, por municípios'
)

# Ajustando as margens
mapa_notas.update_layout(margin={'r':0,'t':0,'l':0, 'b':0})

# Reduzindo a largura das bordas dos municípios
mapa_notas.update_traces(marker_line_width=0.01)

# Exibindo o mapa
mapa_notas.show()

In [ ]:
# Mapa: Municípios, escolas públicas, nota
mapa_notas = px.choropleth_mapbox(
    data_frame = microdados_geo_mun_pub_agg, 
    geojson = geo_cidades, 
    locations='id', 
    featureidkey='properties.id',
    color='nota_media_geral',
    #color_continuous_scale= 'reds',
    range_color=(140, 680),
    mapbox_style='open-street-map',
    zoom=2.5, 
    center = {"lat": -17.14, "lon": -57.33},
    opacity=1,
    labels={'nota_media_geral':'Nota Média, incluindo redação', 
            'id' : 'Código do município'
    },
    width = 1000,
    height = 800,
    title = 'Nota média no ENEM. 2022, por municípios'
)

# Ajustando as margens
mapa_notas.update_layout(margin={'r':0,'t':0,'l':0, 'b':0})

# Reduzindo a largura das bordas dos municípios
mapa_notas.update_traces(marker_line_width=0.01)

# Exibindo o mapa
mapa_notas.show()

###### -> Top 100

In [ ]:
top_100 = microdados_.sort_values('nota_media_geral', ascending=False)[['SG_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'Q006', 'TP_COR_RACA', 'TP_SEXO']].head(100)
top_100.head()


In [ ]:
top_100['TP_COR_RACA'].value_counts(normalize=True)

In [ ]:
top_100['TP_SEXO'].value_counts(normalize=True)

In [ ]:
top_100['Q006'].value_counts(normalize=True)

In [ ]:
top_100['TP_LOCALIZACAO_ESC'].value_counts(normalize=True)

In [ ]:
top_100['TP_DEPENDENCIA_ADM_ESC'].value_counts(normalize=True)

In [ ]:
top_100['SG_UF_ESC'].value_counts(normalize=True)

###### -> Tail 100

In [ ]:
tail_100 = microdados_.sort_values('nota_media_geral', ascending=False)[['SG_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'Q006', 'TP_COR_RACA', 'TP_SEXO']].tail(100)
tail_100.head()


In [ ]:
tail_100['TP_COR_RACA'].value_counts(normalize=True)

In [ ]:
tail_100['TP_SEXO'].value_counts(normalize=True)

In [ ]:
tail_100['Q006'].value_counts(normalize=True)

In [ ]:
tail_100['TP_LOCALIZACAO_ESC'].value_counts(normalize=True)

In [ ]:
tail_100['TP_DEPENDENCIA_ADM_ESC'].value_counts(normalize=True)

In [ ]:
tail_100['SG_UF_ESC'].value_counts(normalize=True)

In [ ]:
microdados_.to_pickle('microdados_.pkl')

In [ ]:
microdados_geo_mun.to_pickle('microdados_geo_mun.pkl')
